### Function
- load dataStruct from NAS
- saved it to "D:\Neural-Pipeline\data\processed"

In [ ]:
def transfertonpz(subject="zarya", date="20250523"):

    import numpy as np
    import h5py
    import os
    from pathlib import Path
    
    base_path = Path(r"\\172.30.3.33\homes\fetschlab\labMembers\Yueh-Chen") / subject / "Neural data"
    file_name = f"@{subject}_neurodata_cleaned.mat"
    matfile_path = base_path / file_name
    output_dir = r"D:\Neural-Pipeline\data\processed"

    target_date = f"{date[:4]}-{date[4:6]}-{date[6:8]}"
    print(f"Target date: {target_date}")

    f = h5py.File(matfile_path, 'r')
    dataStruct = f['dataStruct']

    dates = dataStruct['date']
    session_idx = None

    for i in range(dates.shape[1]):
        try:
            date_ref = dates[0, i]
            
            try:
                session_date = f[date_ref][()].decode('utf-16le').strip()
            except:
                try:
                    session_date = f[date_ref][()].tobytes().decode('utf-16le').strip()
                except:
                    session_date = str(f[date_ref][()]).strip()
            
            if session_date == target_date:
                session_idx = i
                print(f"Found matching session: {session_idx}")
                break
                
        except Exception as e:
            print(f"Error reading date from session {i}: {e}")
            continue

    if session_idx is None:
        print(f"No session found for date {target_date}")


    # Load data from the found session - fix the indexing
    print("Loading data structures...")
    data_refs = dataStruct['data']
    session_data_ref = data_refs[0, session_idx]
    session_data = f[session_data_ref]

    print(f"Session data keys: {list(session_data.keys())}")

    # Process dots3DMP - it's already a group, no need for [0,0]
    print("Processing dots3DMP...")
    dots3DMP = session_data['dots3DMP']

    # Load events, unit, and spike rate references for dots3DMP
    events = dots3DMP['events']
    unit = dots3DMP['unit']
    data_spkrate = dots3DMP['data_spkrate']
   

    print("Loading spike rate matrices for dots3DMP...")
    stimOn_spkrate = []
    # saccOnset_spkrate = []
    # postTargHold_spkrate = []

    stimOn_refs = data_spkrate['stimOn']
    # saccOnset_refs = data_spkrate['saccOnset'] 
    # postTargHold_refs = data_spkrate['postTargHold']

    # print(f"Loading {stimOn_refs.shape} spike rate cells...")
    # for i in range(stimOn_refs.shape[0]):
    #     trial_stimOn = []
    #     trial_saccOnset = []
    #     trial_postTargHold = []
        
    #     for j in range(stimOn_refs.shape[1]):
    #         if stimOn_refs[i, j] != 0:
    #             trial_stimOn.append(f[stimOn_refs[i, j]][:])
    #         else:
    #             trial_stimOn.append(np.array([]))

                
    #         if saccOnset_refs[i, j] != 0:
    #             trial_saccOnset.append(f[saccOnset_refs[i, j]][:])
    #         else:
    #             trial_saccOnset.append(np.array([]))
                
    #         if postTargHold_refs[i, j] != 0:
    #             trial_postTargHold.append(f[postTargHold_refs[i, j]][:])
    #         else:
    #             trial_postTargHold.append(np.array([]))
        
        # stimOn_spkrate.append(trial_stimOn)
    #     saccOnset_spkrate.append(trial_saccOnset)
    #     postTargHold_spkrate.append(trial_postTargHold)
        
    #     if i % 10 == 0:
    #         print(f"  Loaded {i}/{stimOn_refs.shape[0]} units")

    # print("Loading behavioral data for dots3DMP...")
    # choice = events['choice']
    # PDW = events['PDW']
    # modality = events['modality']
    # headingInd = events['headingInd']
    # coherenceInd = events['coherenceInd']
    # goodtrial = events['goodtrial']
    # deltaInd = events['deltaInd']
    # correct = events['correct']
    # oneTargChoice = events['oneTargChoice']
    # oneTargConf = events['oneTargConf']
    # heading = events['heading']
    # coherence = events['coherence']
    # delta = events['delta']
    # RT = events['RT']

    print("Loading unit data...")
    depth = unit['depth']
    cluster_id = unit['cluster_id']

    cluster_group_data = unit['cluster_group']
    group_ref = cluster_group_data['group']


    cluster_group = []
    for i in range(group_ref.shape[1]):  
        # Get the character codes for this unit (column i)
        char_codes = group_ref[:, i]
        
        # Convert to string
        try:
            # Convert uint16 array to string
            group_str = ''.join([chr(code) for code in char_codes if code != 0]).strip()
            cluster_group.append(group_str)
        except Exception as e:
            print(f"Error at unit {i}: {e}")
            cluster_group.append("unknown")

    cluster_group = np.array(cluster_group)

    # # Save dots3DMP
    # output_file_dots3DMP = os.path.join(output_dir, f"{subject}{date}dots3DMP_processed.npz")
    # print(f"Saving dots3DMP to {output_file_dots3DMP}...")

    # np.savez_compressed(output_file_dots3DMP,
    #     stimOn_spkrate=stimOn_spkrate,
    #     saccOnset_spkrate=saccOnset_spkrate, 
    #     postTargHold_spkrate=postTargHold_spkrate,
    #     choice=choice,
    #     PDW=PDW,
    #     modality=modality,
    #     headingInd=headingInd,
    #     coherenceInd=coherenceInd,
    #     goodtrial=goodtrial,
    #     deltaInd=deltaInd,
    #     correct=correct,
    #     oneTargChoice=oneTargChoice,
    #     oneTargConf=oneTargConf,
    #     heading=heading,
    #     coherence=coherence,
    #     delta=delta,
    #     RT=RT,
    #     depth=depth,
    #     cluster_id=cluster_id,
    #     cluster_group=cluster_group,
    #     subject=subject,
    #     date=date,
    #     session_idx=session_idx
    # )

    # Process dots3DMPtuning
    print("Processing dots3DMPtuning...")
    dots3DMPtuning = session_data['dots3DMPtuning']

    events_tuning = dots3DMPtuning['events']
    data_spkrate_tuning = dots3DMPtuning['data_spkrate']

    print("Loading spike rate data for dots3DMPtuning...")
    spkrate_refs = data_spkrate_tuning

    # First pass: determine dimensions
    max_time_bins = 0
    for i in range(spkrate_refs.shape[0]):
        for j in range(spkrate_refs.shape[1]):
            ref = spkrate_refs[i, j]
            if ref != 0:
                data = f[ref][:]
                max_time_bins = max(max_time_bins, len(data))
     

    # Create array in format [event, unit, timeaxis, 1]
    n_events = spkrate_refs.shape[1] 
    n_units = spkrate_refs.shape[0]   
    spkrate = np.zeros((n_events, n_units, max_time_bins, 1))

    for i in range(n_units):
        for j in range(n_events):
            ref = spkrate_refs[i, j]
            if ref != 0:
                data = f[ref][:]
                data = np.squeeze(data)
                spkrate[j, i, :, 0] = data
            else:
                spkrate[j, i, :, 0] = np.nan
        if i % 10 == 0:
            print(f"  Loaded {i}/{n_units} units")

    print(f"Final spkrate shape [event, unit, timeaxis, 1]: {spkrate.shape}")

    print("Loading behavioral data for dots3DMPtuning...")
    goodtrial_tuning = events_tuning['goodtrial']
    headingInd_tuning = events_tuning['headingInd']
    modality_tuning = events_tuning['modality']
    coherenceInd_tuning = events_tuning['coherenceInd']
    deltaInd_tuning = events_tuning['deltaInd']

    # Save dots3DMPtuning
    output_file_tuning = os.path.join(output_dir, f"{subject}{date}dots3DMPtuning_processed.npz")
    print(f"Saving dots3DMPtuning to {output_file_tuning}...")

    np.savez_compressed(output_file_tuning,
        spkrate=spkrate,
        goodtrial=goodtrial_tuning,
        headingInd=headingInd_tuning,
        modality=modality_tuning,
        coherenceInd=coherenceInd_tuning,
        deltaInd=deltaInd_tuning,
        depth=depth,
        cluster_id=cluster_id,
        cluster_group=cluster_group,
        subject=subject,
        date=date,
        session_idx=session_idx
    )

    print("Both files saved successfully!")
    f.close()

### Main

In [12]:
subject = "zarya"
dates = ["20250523", "20250602", "20250702", "20250710"]

for idx, date in enumerate(dates):  
    transfertonpz(subject, date)  

Target date: 2025-05-23
Found matching session: 19
Loading data structures...
Session data keys: ['dots3DMP', 'dots3DMPtuning']
Processing dots3DMP...
Loading spike rate matrices for dots3DMP...
Loading unit data...
Processing dots3DMPtuning...
Loading spike rate data for dots3DMPtuning...
Final spkrate shape [event, unit, timeaxis, 1]: (165, 722, 134, 1)
Loading behavioral data for dots3DMPtuning...
Saving dots3DMPtuning to D:\Neural-Pipeline\data\processed\zarya20250523dots3DMPtuning_processed.npz...
Both files saved successfully!
Target date: 2025-06-02
Found matching session: 20
Loading data structures...
Session data keys: ['dots3DMP', 'dots3DMPtuning']
Processing dots3DMP...
Loading spike rate matrices for dots3DMP...
Loading unit data...
Processing dots3DMPtuning...
Loading spike rate data for dots3DMPtuning...
Final spkrate shape [event, unit, timeaxis, 1]: (162, 528, 134, 1)
Loading behavioral data for dots3DMPtuning...
Saving dots3DMPtuning to D:\Neural-Pipeline\data\processe